# Кошки против собак



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
from tqdm.auto import tqdm

In [ ]:
import torch
import torchvision

import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms as T
from torchvision.datasets import ImageFolder

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: filfonul to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Данные

На этой паре мы обучим нейросетку отличать кошек от собак. Если вы ещё не забыли лекцию про свёрточные сетки, в $2013$ году это было непростой задачкой. Настолько, что [к соревнованию на Kaggle](https://www.kaggle.com/c/dogs-vs-cats) висит такое предысловие:

> В 1997 году Deep Blue обыграл в шахматы Каспарова.  В 2011 Watson обставил чемпионов Jeopardy. Сможет ли ваш алгоритм в 2013 году отличить Бобика от Пушистика?

В этом семинаре мы попробуес сделать Transfer learning и посмотрим какое качество будет у нашей модели.

In [ ]:
%%bash
wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
%%bash
unzip -q cats_and_dogs_filtered.zip
rm cats_and_dogs_filtered.zip
rm cats_and_dogs_filtered/vectorize.py

In [ ]:
!du -sh cats_and_dogs_filtered

72M	cats_and_dogs_filtered


Все файлы сейчас лежат по папкам, которые соотвествуют классам.

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

In [ ]:
path = !pwd
path = path[0] + '/'
path

'/content/'

In [ ]:
normalize = T.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    normalize,
])

aug_transform = T.Compose([
    T.ColorJitter(hue=0.05, saturation=0.05),
    T.RandomHorizontalFlip(),
    T.RandomRotation(20),
    T.Resize(256),
    T.RandomResizedCrop(224, scale=(0.5, 1.0)),
    T.ToTensor(),
    normalize,
])

dataset_pets = ImageFolder(
    path +  'cats_and_dogs_filtered/train/',
    transform = aug_transform #test_transform
 )

# создаём встроенными методами два датасета :)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset_pets, [int(0.8 * len(dataset_pets)), len(dataset_pets) - int(0.8 * len(dataset_pets))]
)

test_dataset = ImageFolder(
    path +  'cats_and_dogs_filtered/validation/',
    transform = test_transform
 )

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size     = 32,
    shuffle        = True,
    pin_memory     = True,
    num_workers    = 0
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size    = 4096,
    shuffle       = False,
    pin_memory     = True,
    num_workers   = 0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size    = 4096,
    shuffle       = False,
    pin_memory     = True,
    num_workers   = 0
)

In [ ]:
dataset_pets.classes

['cats', 'dogs']

In [ ]:
image, label = next(iter(dataset_pets))
dataset_pets.classes[label]

image.shape

torch.Size([3, 224, 224])

In [ ]:
# print(image.shape)
# plt.imshow(np.transpose(image, (1,2,0)));

In [ ]:
len(dataset_pets)

2000

## 2. Циклы для обучения

In [ ]:
from IPython.display import clear_output

def plot_losses(train_losses, test_losses, train_accuracies, test_accuracies):
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(13, 4))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(test_losses) + 1), test_losses, label='test')
    axs[0].set_ylabel('loss')

    axs[1].plot(range(1, len(train_accuracies) + 1), train_accuracies, label='train')
    axs[1].plot(range(1, len(test_accuracies) + 1), test_accuracies, label='test')
    axs[1].set_ylabel('accuracy')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    plt.show()

In [ ]:
def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc, wandb_project=None):
    """Одна эпоха обучения"""
    train_loss, train_accuracy = 0.0, 0.0
    model.train()
    for images, labels in tqdm(train_loader, desc=tqdm_desc):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.shape[0]
        train_accuracy += (logits.argmax(dim=1) == labels).sum().item()

        if wandb_project:
            metrics = {
                "batch-train/loss": loss.item()
            }
            wandb.log(metrics)

    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    return train_loss, train_accuracy


@torch.no_grad()
def validation_epoch(model, criterion, val_loader, tqdm_desc):
    """Прогнозы на валидации"""

    val_loss, val_accuracy = 0.0, 0.0
    model.eval()
    for images, labels in tqdm(val_loader, desc=tqdm_desc):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        val_loss += loss.item() * images.shape[0]
        val_accuracy += (logits.argmax(dim=1) == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy /= len(val_loader.dataset)
    return val_loss, val_accuracy


def train(
    model, optimizer, criterion, train_loader, val_loader, num_epochs, scheduler=None,
    wandb_project=None, config=None
    ):
    """Обучение модели"""
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    # подключаем wandb
    if wandb_project:
        wandb.init(
            project=wandb_project,
            config=config
        )
        wandb.watch(model)

    for epoch in range(1, num_epochs + 1):
        clear_output()
        train_loss, train_accuracy = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}',
            wandb_project=wandb_project
        )
        val_loss, val_accuracy = validation_epoch(
            model, criterion, val_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}'
        )

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # функция для смены lr по расписанию
        if scheduler is not None:
            scheduler.step()

        if wandb_project:
            metrics = {
                "train/loss": train_loss / len(train_loader.dataset),
                "train/accuracy": train_accuracy / len(train_loader.dataset),
                "val/loss": val_loss,
                "val/accuracy": val_accuracy
            }
            wandb.log(metrics)
        else:
          plot_losses(train_losses, val_losses, train_accuracies, val_accuracies)

        plt.show()
    # печатаем метрики
    print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}, accuracy: {np.mean(val_accuracy)}")

## 3. Модели

In [ ]:
class ConvNet(nn.Module):

    def __init__(self, image_channels=3):
        super().__init__()

        self.encoder = nn.Sequential(  # 28 x 28
            nn.Conv2d(in_channels=image_channels, out_channels=4,
                      kernel_size=3, padding='same'),  # 28 x 28
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14 x 14

            nn.Conv2d(in_channels=4, out_channels=8,
                      kernel_size=3, padding='same'),  # 14 x 14
            nn.ReLU(),
            nn.MaxPool2d(2),  # 7 x 7

            nn.Conv2d(in_channels=8, out_channels=16,
                      kernel_size=3, padding='same'),  # 7 x 7
            nn.ReLU(),
            nn.MaxPool2d(2)  # 3 x 3
        )

        self.head = nn.Sequential(
            nn.LazyLinear(out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=10)
        )

    def forward(self, x):
        # x: B x 1 x 28 x 28
        out = self.encoder(x)   # out: B x 392
        out = nn.Flatten()(out) # out: B x 128
        out = self.head(out)    # out: B x 10
        return out

    def get_embedding(self, x):
        out = self.encoder(x)
        return nn.Flatten()(out)

In [ ]:
model_cnn = ConvNet( )

## 4. Обучение

In [ ]:
x_batch, y_batch = next(iter(train_loader))
x_batch.shape

torch.Size([32, 3, 224, 224])

In [ ]:
NUM_EPOCH = 10
LR = 0.01

model_cnn = ConvNet().to(device)
model_cnn(x_batch.to(device)) # костыль для lazy init

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=LR, momentum=0)

config_model = {
    "architecture": 'CNN',
    "learning_rate": LR,
    "scheduler": 'None',
    "epochs":  NUM_EPOCH,
    "optimizer": 'SGD',
    "experiment": 'Augmented CNN'
}

train(model_cnn, optimizer, criterion, train_loader, val_loader, NUM_EPOCH,
      wandb_project='cats_vs_dogs', config = config_model
);

Training 10/10:   0%|          | 0/50 [00:00<?, ?it/s]

Validating 10/10:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 10, loss: 0.6905878186225891, accuracy: 0.5425


In [ ]:
NUM_EPOCH = 10
LR = 0.001

model_cnn = ConvNet().to(device)
model_cnn(x_batch.to(device)) # костыль для lazy init

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn.parameters())

config_model = {
    "architecture": 'CNN',
    "learning_rate": LR,
    "scheduler": 'None',
    "epochs":  NUM_EPOCH,
    "optimizer": 'Adam',
    "experiment": 'Augmented CNN'
}

train(model_cnn, optimizer, criterion, train_loader, val_loader, NUM_EPOCH,
      wandb_project='cats_vs_dogs', config = config_model
);

Training 10/10:   0%|          | 0/50 [00:00<?, ?it/s]

Validating 10/10:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 10, loss: 0.6300007104873657, accuracy: 0.6525


In [ ]:
NUM_EPOCH = 10
LR = 0.001

model_cnn = ConvNet().to(device)
model_cnn(x_batch.to(device)) # костыль для lazy init

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_cnn.parameters())

config_model = {
    "architecture": 'CNN',
    "learning_rate": LR,
    "scheduler": 'None',
    "epochs":  NUM_EPOCH,
    "optimizer": 'AdamW',
    "experiment": 'Augmented CNN'
}

train(model_cnn, optimizer, criterion, train_loader, val_loader, NUM_EPOCH,
      wandb_project='cats_vs_dogs', config = config_model
);

### Различные трюки:

1. Начните с маленькой сети. Не забывайте прикидывать сколько наблдюдений  тратится на оценку каждого из  параметров. Если величина очень маленькая, не забывайте о регуляризации.
2. Всегда оставляйте часть выборки под валидацию на каждой эпохе.
3. Усложняйте модель, пока качество на валидации не начнёт падать.
4. Не забывайте проскалировать ваши наблюдения для лучшей сходимости.
5. Можно попробовать ещё целую серию различных трюков:
  - Архитектура нейросети
    - Больше/меньше нейронов
    - Больше/меньше слоёв
    - Другие функции активации (tanh, relu, leaky relu, elu etc)
    - Регуляризация (dropout, l1,l2)
  - Более качественная оптимизация
    - Можно попробовать выбрать другой метод оптимизации
    - Можно попробовать менять скорость обучения, моментум и др.
    - Разные начальные значения весов
  - Попробовать собрать больше данных
  - Для случая картинок объёмы данных можно увеличить искусственно с помощью подхода, который называется Data augmemntation

И это далеко не полный список. Обратите внимание, что делать grid_search для больших сеток это довольно времязатратное занятие...

Логгируйте свои эксперименты. За один прогон пробуйте одно изменение. Иначе будет непонятно какие именно изменения улучшили качество, а какие ухудшили.